# Breaking into error mitigation

Things that it gets covered:
- What error mitigation technique should I use? 
Getting started with Mitiq's calibration module.
- Tutorial on a simple use case: Randomized Benchmarking circuits
- Run with default ZNESettings  
- Explore results with settings changes
- Conclusion

In [2]:
import cirq
import numpy as np
from mitiq import MeasurementResult

In [3]:
def execute(circuit, noise_level=0.001):
    circuit = circuit.with_noise(cirq.amplitude_damp(noise_level))
    result = cirq.DensityMatrixSimulator().run(circuit, repetitions=100)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

In [4]:
from mitiq.calibration import Calibrator, ZNESettings, execute_with_mitigation

In [5]:
cal = Calibrator(execute, ZNESettings)
print(cal.get_cost())
cal.run()

## could add print message from ``run_circuits(self)`` if a `verbose=True` option is verified.



{'noisy_executions': 24, 'ideal_executions': 0}


/Users/nathanshammah/github/mitiq/mitiq/zne/inference.py:107: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nathanshammah/github/mitiq/mitiq/zne/inference.py:107: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nathanshammah/github/mitiq/mitiq/zne/inference.py:107: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nathanshammah/github/mitiq/mitiq/zne/inference.py:107: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nathanshammah/github/mitiq/mitiq/zne/inference.py:107: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nathanshammah/github/mitiq/mitiq/zne/inference.py:107: UserWarning: The input circuit is very short. This may reduce the accuracy of 

In [25]:
from mitiq.benchmarks import generate_rb_circuits

circuit = generate_rb_circuits(2, 10)[0]#,trials=3)#[0]
print(len(circuit))

77


In [27]:
circuit

0: ───X───────Y^0.5───@───Y^-0.5───@───Y^0.5────@───Y^-0.5───X^0.5───Y^-0.5───@───Y^0.5────@───Y───────X^0.5───X───────Y^0.5────────────@───Y^0.5────X^0.5────Y^-0.5───X^0.5────Y^-0.5───────────@───X^0─────X^-0.5───Y^0─────@───Y^-0.5───@───Y^0.5────@───X^0.5───Y^-0.5───────────@───Y^0.5────@───Y────────X^0.5───X────────Y^0.5───@───Y^-0.5───@───Y^0.5────@───Y^-0.5───X^-0.5───────────────────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5───X───Y^-0.5───@───X^-0.5───Y^-0.5───Y^0.5────X^0.5───@───Y^0.5────@───Y───────X^0.5───Y^-0.5───X^0.5──────────@───X^-0.5───Y^-0.5───
                      │            │            │                             │            │                                            │                                                        │                            │            │            │                            │            │                                     │            │            │                                     │            │                                          │                                      │            │                                           │
1: ───Y^0.5───X^0─────@───Y^0.5────@───Y^-0.5───@───Y^0.5────Y^0.5───X^-0.5───@───X^-0.5───@───X^0.5───Y^0.5───X^0.5───Y───────X^-0.5───@───X^-0.5───Y^-0.5───X^0.5────Y^-0.5───X^-0.5───Y^0.5───@───Y^0.5───Y^0.5────X^0.5───@───Y^0.5────@───Y^-0.5───@───Y^0.5───X^-0.5───Y^0.5───@───X^-0.5───@───Y^-0.5───Y───────X^-0.5───────────@───Y^0.5────@───Y^-0.5───@───Y^0.5────Y^0.5────X^0.5───Y^0.5───@───X^-0.5───@───Y^-0.5───X^0.5────Y^0──────────────────@───Y^0.5────Y^-0.5───X^-0.5───────────@───X^-0.5───@───X^0.5───Y^0.5───X^0.5────Y^-0.5───X^0───@───Y^0.5─────────────

In [28]:
# circuits passed to an executor returning bitstrings must contain measurements
circuit.append(cirq.measure(circuit.all_qubits()))

In [9]:
def execute(circuit, noise_level=0.001):
    circuit = circuit.with_noise(cirq.amplitude_damp(noise_level))

    rho = (
        cirq.DensityMatrixSimulator()
        .simulate(circuit)
        .final_density_matrix
    )
    return rho[0, 0].real

execute_with_mitigation(circuit, execute, calibrator=cal)

1.4166545675009456

In [31]:
from mitiq.zne import *
def execute(circuit, noise_level=0.001):
    circuit = circuit.with_noise(cirq.amplitude_damp(noise_level))

    rho = (
        cirq.DensityMatrixSimulator()
        .simulate(circuit)
        .final_density_matrix
    )
    return rho[0, 0].real

execute_with_zne(circuit, execute)

0.9999978542327875